In [1]:
import pickle
from utils import figure_utils
import copy
import pandas as pd

### Load the ME model (updated) & solution, and COGs

In [2]:
with open("run_stressme_results/demo/StressME_heatevolved_T_37_pH_7.0_ROS_1.0X_sol.pickle", "rb") as f:
    solution = pickle.load(f)

with open("run_stressme_results/demo/demo_updated_model.pickle", "rb") as f:
    me = pickle.load(f)

# Now you can use the 'me' model object
print(me)

# Now you can use the 'solution' object
print(solution.x_dict['biomass_dilution'], solution.status)


#COG dataframe
cogs_file = "/source/ecolime/ecolime/characterization/data/cogs_ecoli_mg1655.csv"
cog_df = pd.read_csv(cogs_file,encoding="ISO-8859-1")
cog_df = cog_df.set_index('locus')
cog_df.head()

/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


iJL1678b-ME
0.70556640625 optimal


,GI,gene,COG ID,COG category,COG name,COG category primary,COG class,COG description
locus,,,,,,,,
b0002,16127996,thrA,COG0460,E,Homoserine dehydrogenase,E,METABOLISM,Amino acid transport and metabolism
b3940,16131778,metL,COG0460,E,Homoserine dehydrogenase,E,METABOLISM,Amino acid transport and metabolism
b0002,16127996,thrA,COG0527,E,Aspartokinase,E,METABOLISM,Amino acid transport and metabolism
b3940,16131778,metL,COG0527,E,Aspartokinase,E,METABOLISM,Amino acid transport and metabolism
b4024,16131850,lysC,COG0527,E,Aspartokinase,E,METABOLISM,Amino acid transport and metabolism


### Identify translation fluxes

In [17]:
# Get all translation reactions with flux > 0
translation_fluxes = {rxn: [float(flux)] for rxn, flux in solution.x_dict.items() if "translation_" in rxn and flux > 0}
results = {'rxn_flux' :translation_fluxes, 'time' :[0], "biomass" : [1]}

In [18]:
# Export weights at t = 6.0 h by absolute rate (mmol L^-1 hr^-1)
df_weights = figure_utils.export_translation_weights(
    me,
    results,
    cog_df=cog_df,
    t_query=0, # select which time point you want
    value="mass", #(g L^-1 hr^-1) or "flux" (mmol gDW^-1 hr^-1), "rate" (mmol L^-1 hr^-1)
    out_file=None #path to save the file compatible with Proteomaps 

)
df_weights["COG category"] = df_weights["COG category"].fillna("Unknown")

print ("These genes do not completely match up to the coloring scheme:")
df_weights[df_weights.get('COG category').isin(figure_utils.cog_colors.keys()) == False].head()

These genes do not completely match up to the coloring scheme:


,COG category,gene,locus,weight
236,CP,nuoL,b2278,0.004066
76,HI,dxs,b0420,0.002405
482,EG,ilvD,b3771,0.001866
481,EH,ilvE,b3770,0.000989
496,HC,fre,b3844,0.000419


In [16]:
hier = figure_utils.build_hierarchy_for_d3_from_df(
    df_weights, 
    cog_colors=figure_utils.cog_colors,
    label_col="gene" # or 'locus' or modify df_weights with peronsalized column (i.e. gene product, uniprot id)
)
figure_utils.write_d3_voronoi_html(hier, out_html="run_stressme_results/demo/voronoi_acetate.html", cog_colors = figure_utils.cog_colors)


✅ Wrote: run_stressme_results/demo/voronoi_acetate.html
